In [1]:
from datetime import datetime, timedelta
from pandas import DataFrame
from models.Payload import REGION, RegPayload, SecPayload
from models.Source import DataSource as SecApi
from utils.transforms import str2date
from urllib.parse import urlencode as url
#cspell:disable

#input
region:REGION='Valparaiso'
#total regional clients
users = SecApi()
q = users.request('reg_clients',RegPayload(region=region))
clients = int(q[0]['CLIENTES'])
print(f"usuarios regionales: {clients:,.0f} clientes")

#data
curr = SecApi()
curr.request('affected_detail',SecPayload()) 
curr_df=curr.dataframe(FECHA_INT_STR=str2date)
curr_df= curr_df[curr_df['NOMBRE_REGION']==region].drop(columns=['HORA','DIA','MES','ANHO'])
affected = curr_df['CLIENTES_AFECTADOS'].sum()

print('usuarios sin servicio :',f'{affected:,.0f}',' clientes',f'{affected/clients:.2%}')

res:DataFrame = curr_df.groupby(['NOMBRE_COMUNA','NOMBRE_EMPRESA','FECHA_INT_STR'],as_index=False)['CLIENTES_AFECTADOS'].sum().sort_values(['CLIENTES_AFECTADOS'],ascending=False)

#cspell:disable
res:DataFrame = res.rename(columns={
    'FECHA_INT_STR':'INCIDENTE',
    'CLIENTES_AFECTADOS':'SIN_SERVICIO',
    'NOMBRE_EMPRESA':'DISTRIBUIDORA',
    'NOMBRE_COMUNA':'COMUNA'
    })

#days w/o service
res['TIEMPO_SIN_SERVICIO'] = res['INCIDENTE'].apply(lambda event_date: datetime.now()-event_date+timedelta(days=1) )
res

usuarios regionales: 780,563 clientes


TypeError: '>' not supported between instances of 'NoneType' and 'int'

In [ ]:
res_by_city:DataFrame = curr_df.groupby(['NOMBRE_COMUNA'],as_index=False)['CLIENTES_AFECTADOS']\
    .sum().sort_values(['CLIENTES_AFECTADOS'],ascending=False)
res_by_city

In [ ]:
def test(**kwargs:dict[str,int]):
    print(kwargs)

test(hours=10,days=20)